# Intro to BigQuery and SQL

In [2]:
from google.cloud import bigquery

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/levka/Downloads/KaggleSQL-79493a7efc0a.json"

In [13]:
# Create a "Client" object 
client = bigquery.Client()

In BigQuery, each dataset is contained in a corresponding project. In this case, your hacker_news dataset is contained in the bigquery-public-data project.

To access the dataset:

- We begin by constructing a reference to the dataset with the dataset() method.

- Next, we use the get_dataset() method, along with the reference we just constructed, to fetch the dataset.

In [17]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project = "bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the list_tables() method to list the tables in the dataset.

In [18]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print the names of all tables in the dataset
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


Similar to how we fetched a dataset, we can fetch a table. In the code cell below we fetch the full table in the hacker_news dataset

In [19]:
# COnstruct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

![alt text](BigQuery.PNG "BigQuery")

## Table schema

The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.

In [20]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('ran

Each SchemaField tells us about a specific column (which we also refer to as *field*). In order, the information is:

- The *name* of the column

- The *field type* (or datatype) in the column

- The *mode* of the column ('NULLABLE' means that a column allows NULL values, and is the default)

- A *description* of the data in that column

The first field has the SchemaField:

SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())

this means:

- the field (or column) is called by,

- the data in this field is strings,

- NULL values are allowed, and

- it contains the usernames corresponding to each item's author

We can use the list_rows() method to check just the first five lines of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery RowIterator object that can quickly be converted to a pandas DataFrame with the to_dataframe() method.



In [22]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results = 5).to_dataframe()

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,us0r,NaN,1514138229,2017-12-24 17:57:09+00:00,,comment,,thats not even close to the final price. they...,15999907.0,None,None,NaN,16000479,None
1,dagobertus79,1.0,1312718363,2011-08-07 11:59:23+00:00,Herzlich Willkommen bei bis zu 12 Casino-Tricks,story,http://www.4jetons.eu/,,NaN,None,True,-1.0,2856453,None
2,mherrmann,NaN,1542720628,2018-11-20 13:30:28+00:00,,comment,,You did read the section &quot;Maybe you also ...,18493612.0,None,None,NaN,18493616,None
3,mdadm,NaN,1469750768,2016-07-29 00:06:08+00:00,,comment,,From your link:<p>&gt;Applies to: Office 2007 ...,12182633.0,None,None,NaN,12184180,None
4,narrowingorbits,NaN,1494967212,2017-05-16 20:40:12+00:00,,comment,,"As I said, that makes sense, and I agree. That...",14352955.0,None,None,NaN,14353117,None


The list_rows() method will also let us look at just the information in a specific column. If we want to see the first five entries in the by column, for example we can do that:

In [30]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields = table.schema[:1], max_results = 5).to_dataframe()

,by
0,us0r
1,dagobertus79
2,mherrmann
3,mdadm
4,narrowingorbits


## Exercise

### Introduction

The first test of your new data exploration skills uses data describing crime in the city of Chicago.



In [4]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)


### Count tables in the dataset

How many tables are in the Chicago Crime dataset?

In [10]:
tables = list(client.list_tables(dataset))
tables_names = [x.table_id for x in list(client.list_tables(dataset))]
print(len(tables_names))

1


In [11]:
print(tables_names)

['crime']


### Explore the table schema

How many columns in the `crime` table have `TIMESTAMP` data?

In [12]:
# COnstruct a reference to the "crime" table
table_ref = dataset_ref.table("crime")

# API request - fetch the table
table = client.get_table(table_ref)

In [25]:
len([ c for c in table.schema if c.field_type == "TIMESTAMP"])

2

### Create a crime map

If you wanted to create a map with a dot at the location of each crime, what are the names of the two fields you likely need to pull out of the `crime` table to plot the crimes on a map?

In [26]:
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', ()),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', ()),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', ()),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', ()),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', ()),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', ()),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a subcategory of the primary descriptio

I would need latitude and longitude.


Thinking about the question above, there are a few columns that appear to have geographic data. Look at a few values (with the `list_rows()` command) to see if you can determine their relationship.  Two columns will still be hard to interpret. But it should be obvious how the `location` column relates to `latitude` and `longitude`.

In [28]:
client.list_rows(table, max_results = 5).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11725358,JC309736,2019-06-15 00:00:00+00:00,058XX N ST JOHNS CT,1156,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,OTHER,False,False,...,39,12,11,1146654.0,1938708.0,2019,2019-06-30 15:56:27+00:00,41.987793,-87.735964,"(41.987792784, -87.735963789)"
1,11719073,JC301927,2019-04-24 22:30:00+00:00,064XX S NARRAGANSETT AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,23,64,02,1134872.0,1860926.0,2019,2019-06-30 15:56:27+00:00,41.774566,-87.781140,"(41.774565637, -87.781140173)"
2,11657538,JC227404,2019-04-16 23:39:00+00:00,018XX W 35TH ST,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,GAS STATION,False,False,...,12,59,14,1164779.0,1881499.0,2019,2019-04-23 16:22:49+00:00,41.830441,-87.670924,"(41.830441145, -87.670923807)"
3,11654695,JC223942,2019-04-14 04:14:00+00:00,098XX S AVENUE G,2028,NARCOTICS,POSS: SYNTHETIC DRUGS,PARK PROPERTY,True,False,...,10,52,18,1203118.0,1840607.0,2019,2019-04-21 16:07:03+00:00,41.717334,-87.531658,"(41.717334211, -87.531658377)"
4,11642355,JC208983,2019-03-30 08:35:00+00:00,074XX W SEMINOLE ST,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,RESIDENCE,False,False,...,41,10,11,1125471.0,1937243.0,2019,2019-04-06 16:04:43+00:00,41.984152,-87.813910,"(41.984151676, -87.813910429)"
